In [1]:
"""
Script to download from Huggingface, create training data, and fine tune model
"""
import os
import torch
import pandas as pd
import random
import wandb
wandb.login()

# Huggingface scripts for fine-tuning models and language generation
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/language-modeling/run_language_modeling.py -q
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/text-generation/run_generation.py -q

path = "quotes.csv"
df = pd.read_csv(path)
quote = "quote"

# shuffle data
random.shuffle(list(df.quote))

# fraction of training data
split_train_valid = 0.9

# split dataset
train_size = int(split_train_valid * len(list(df.quote)))
valid_size = len(list(df.quote)) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(list(df.quote), [train_size, valid_size])


with open('{}_train.txt'.format('quote'), 'w') as f:
    f.write('\n'.join(train_dataset))

with open('{}_valid.txt'.format('quote'), 'w') as f:
    f.write('\n'.join(valid_dataset))

!python3 run_language_modeling.py \
    --output_dir=output/$quote \
    --overwrite_output_dir \
    --model_type=gpt2 \
    --model_name_or_path=gpt2 \
    --do_train --train_data_file=$quote\_train.txt \
    --do_eval --eval_data_file=$quote\_valid.txt \
    --evaluate_during_training \
    --logging_steps 20 \
    --per_gpu_train_batch_size 1 \
    --num_train_epochs 4

The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
07/02/2020 13:28:44 - INFO - transformers.training_args -   PyTorch: setting up devices
07/02/2020 13:28:44 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
07/02/2020 13:28:44 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='output/quote', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=True, per_device_train_batch_size=8, per_device_eval_batch


## Problem: I neeed more data to fine tune each model to the persons speech pattern. 
    
    I will basically be creating NLP philosphy generator
    
    this will result in 24 fine tuned models that represent each person, similar to the twitter models- calculate space cost of models, calculate time to train
    
    there is a trade off present: if I do not care about space I can just create and save a bunch of differnet models
                                  if i do care, I will, train, test, create conversation, and upload in one go
    
    the first option seems straightforward, but the time to train the models could be problamatic, 
        and the space issue means that i am pretty much dealing with a local script that i cannot deploy somewhere.
        i would eventually like to deploy this to a server and merely have a docker container.is this airflow use case?
    
    the second option could be faster, but i need to train one model for the 1st speaker and for each subsequent person
    a marriage between the two will use the straightforward double for loop and check for model for diff users. if it doesnt exist, 
        train on the data for person, run algorithm. 
    
## Problem: I still need an algorithm to generate a dialog between speaker x and speaker y
    this may incorporate different pipelines
        ex: prompt -> generate text -> summarize?, NER?, question answer? -> prompt
    
    a harder question is how to model the sequence format such that it resembles dialog
    
    I can just generate summerize the first speakers quotes, generate text, summerize, use as prompt for 2nd speaker, repeat